In [126]:
import os
import glob
import pandas as pd
import numpy as np
import re
import unicodedata

In [127]:
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9.%/,-]', '', text)

def normalize_text(text):
    """Remove hidden Unicode control characters like \u202d"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

def is_numeric(s):
    return s.apply(lambda x: re.fullmatch(r'\d+', x) is not None).all()

def is_decimal(s):
    return s.apply(lambda x: re.fullmatch(r'\d+\.\d+', x) is not None).all()

def has_special_characters(s):
    return s.apply(lambda x: any(c in x for c in ['/', '-', ',', '.', '%'])).any()

def is_alphanumeric(s):
    return s.apply(lambda x: re.fullmatch(r'[A-Za-z0-9]+', x) is not None).all()

def is_mostly_letters(s, threshold=1.0):
    letter_counts = s.apply(lambda x: sum(c.isalpha() for c in x))
    total_counts = s.apply(lambda x: len(x))
    ratios = letter_counts / total_counts
    return (ratios == threshold).mean() >= 1.0

def fixed_length(s):
    cleaned = s.apply(clean_text)
    lengths = cleaned.apply(len).unique()
    return len(lengths) == 1

In [128]:
def load_input_file(input_directory):
    files = glob.glob(os.path.join(input_directory, "*.xlsx"))
    if not files:
        raise FileNotFoundError("No XLSX file found.")
    file_path = files[0]
    df = pd.read_excel(file_path, skiprows=0)
    df_for_mapping = df.head(10)
    print(f"Loaded input file '{file_path}' with {len(df)} rows. Using top 10 rows for mapping.")
    return df, df_for_mapping


In [129]:
def load_seed_files(seed_directory):
    seed_files = glob.glob(os.path.join(seed_directory, "*.csv"))
    seeds = {}
    for file in seed_files:
        df = pd.read_csv(file, skipinitialspace=True, dtype=str)
        df.columns = df.columns.str.strip()   
        filename = os.path.splitext(os.path.basename(file))[0]
        seeds[filename] = df
        print(f"Loaded seed file: '{filename}' with {len(df)} rows.")
    return seeds


In [130]:
def find_primary_keys(df, tolerance=0):
    primary_keys = []
    for col in df.columns:
        s = df[col].dropna().astype(str).apply(normalize_text).apply(clean_text)
        if s.empty:
            continue
        total_count = len(s)
        unique_count = len(s.unique())
        
        if total_count - unique_count <= tolerance:
            primary_keys.append(col)
            # print(f"Primary key candidate: '{col}' (Total: {total_count}, Unique: {unique_count})")
        else:
            #  Debug print if duplicates found
            # print(f"\nColumn '{col}' not picked as primary key (Total: {total_count}, Unique: {unique_count}).")
            # print(f"Finding duplicates for '{col}'...\n")
            duplicated_values = s[s.duplicated(keep=False)]
            if not duplicated_values.empty:
                print(df.loc[duplicated_values.index, [col]])
            print("-" * 50)

    return primary_keys


In [131]:
def analyze_column_properties(s):
    cleaned_s = s.dropna().astype(str).apply(normalize_text).apply(clean_text)
    properties = {
        'is_numeric': is_numeric(cleaned_s),
        'is_decimal': is_decimal(cleaned_s),
        'has_special_characters': has_special_characters(cleaned_s),
        'is_alphanumeric': is_alphanumeric(cleaned_s),
        'is_mostly_letters': is_mostly_letters(cleaned_s),
        'is_fixed_length': fixed_length(cleaned_s),
        'length': cleaned_s.apply(len).unique()[0] if fixed_length(cleaned_s) else None
    }
    return properties

In [132]:
def map_columns(input_sample, seed_col_name, seed_col_values, seed_props):
    matches = []
    for col in input_sample.columns:
        s_input = input_sample[col].dropna().astype(str).apply(normalize_text).apply(clean_text)
        if s_input.empty:
            continue
        cleaned_input = s_input

        input_props = {
            'is_numeric': is_numeric(cleaned_input),
            'is_decimal': is_decimal(cleaned_input),
            'has_special_characters': has_special_characters(cleaned_input),
            'is_alphanumeric': is_alphanumeric(cleaned_input),
            'is_mostly_letters': is_mostly_letters(cleaned_input),
            'is_fixed_length': fixed_length(cleaned_input),
            'length': cleaned_input.apply(len).unique()[0] if fixed_length(cleaned_input) else None
        }

        conditions = all(seed_props[key] == input_props.get(key) for key in seed_props)
        # conditions = all(
        #     seed_props[key] == input_props.get(key)
        #     or (key in ['has_special_characters', 'is_mostly_letters'] and True)
        #     for key in seed_props
        # )

        if conditions:
            print(f"Matched input column '{col}' for seed column '{seed_col_name}'")
            matches.append(col)
    
    if not matches:
        print(f"No match found for seed column '{seed_col_name}'")
    
    return matches

In [133]:
# Step 8: Full processing flow

input_directory = "input"
seed_directory = "seeds"

# Load input file
full_input_df, input_sample = load_input_file(input_directory)

# Load seed files
seeds = load_seed_files(seed_directory)

# Analyze and map
all_mappings = {}

for seed_name, seed_df in seeds.items():
    print(f"\n=== Processing seed file '{seed_name}' ===")
    primary_keys = find_primary_keys(seed_df)

    mappings = []
    for pk_col in primary_keys:
        if pk_col not in seed_df.columns:
            continue

        seed_col_values = seed_df[pk_col]
        seed_props = analyze_column_properties(seed_col_values)

        print(f"\nProperties for seed column '{pk_col}': {seed_props}")
        
        matched_cols = map_columns(input_sample, pk_col, seed_col_values, seed_props)

        if matched_cols:
            mappings.append({
                'seed_column': pk_col,
                'matched_input_columns': matched_cols,
                'seed_properties': seed_props
            })

    all_mappings[seed_name] = mappings


Loaded input file 'input/main_input.xlsx' with 1669 rows. Using top 10 rows for mapping.
Loaded seed file: 'druva_seed' with 644 rows.

=== Processing seed file 'druva_seed' ===
           GSTIN/UID
319  27AAAFW4298E1ZY
320  27ADBFS5581J1ZO
321  29ABOFS7511F1ZU
322  27AAQFV2256H1ZR
323  27AAXFR4592D1ZL
324  27ADEFS7931B1Z4
325  27ADMFS4228H1ZS
326  27AAXFP4248E1ZS
327  27AAEFP1428R1ZS
328  27AAAFK1415H1ZR
329  27AAFFL1052J1ZF
330  27ABIFM0007A1ZZ
331  27AALFK8338E1Z2
332  27AATFK2284M1ZK
333  27AAOFK0606N1Z1
334  24AAPFD2680Q1ZT
335  27AACFD4815A2ZZ
336  27AACFG9740K1Z2
337  24CIKPS6663M1ZG
348  24CIKPS6663M1ZG
371  27ADBFS5581J1ZO
411  27AAEFP1428R1ZS
442  29ABOFS7511F1ZU
484  27ADMFS4228H1ZS
486  27ADEFS7931B1Z4
490  27ABIFM0007A1ZZ
498  27AAXFR4592D1ZL
499  27AAXFP4248E1ZS
505  27AATFK2284M1ZK
509  27AAQFV2256H1ZR
512  27AAOFK0606N1Z1
517  27AALFK8338E1Z2
530  27AAFFL1052J1ZF
549  27AACFG9740K1Z2
550  27AACFD4815A2ZZ
584  27AAAFW4298E1ZY
586  27AAAFK1415H1ZR
613  24AAPFD2680Q1ZT
626

In [134]:
print("\n=== Final Mapping Results ===")
for seed_name, mappings in all_mappings.items():
    for mapping in mappings:
        input_cols = ", ".join(mapping['matched_input_columns'])
        print(f"Seed File '{seed_name}': '{mapping['seed_column']}' -> {input_cols}")


=== Final Mapping Results ===
Seed File 'druva_seed': 'S_no' -> Document Number
Seed File 'druva_seed': 'Name' -> Vendor, Item, Account, Memo, Amount (Trxn Currency) w/ Correct GL Sign, Amount (Base Currency) w/ Correct GL Sign


In [135]:
final_verified_mappings = {}

for seed_name, mappings in all_mappings.items():
    final_verified_mappings[seed_name] = []
    for mapping in mappings:
        seed_column_name = mapping['seed_column']
        full_seed_col_values = seeds[seed_name][seed_column_name].dropna().astype(str).apply(lambda x: x.strip())

        for input_col_name in mapping['matched_input_columns']:
            if input_col_name not in full_input_df.columns:
                continue
            full_input_col_values = full_input_df[input_col_name].dropna().astype(str).apply(lambda x: normalize_text(x.strip()))


            input_unique = set(full_input_col_values)
            seed_unique = set(full_seed_col_values)

            if not input_unique:
                continue

            common_elements = input_unique & seed_unique
            ratio = len(common_elements) / len(input_unique)

            print(f"\nMapping Attempt: '{seed_name}:{seed_column_name}' -> '{input_col_name}'")
            print(f"Input Unique Values ({input_col_name}): {sorted(input_unique)}")
            print(f"Seed Unique Values ({seed_column_name}): {sorted(seed_unique)}")
            print(f"Common Values: {sorted(common_elements)}")
            print(f"Input Unique Count: {len(input_unique)}, Common Count: {len(common_elements)}, Ratio: {ratio:.2f}")

            if ratio >= 0.8:
                print(" Mapping Accepted based on Common Elements Ratio.\n")
                final_verified_mappings[seed_name].append({
                    'seed_column': seed_column_name,
                    'input_column': input_col_name,
                    'ratio': ratio
                })
            else:
                print("Mapping Rejected based on Common Elements Ratio.\n")

print("\n=== Final Verified Mappings ===")
for seed_name, verified_list in final_verified_mappings.items():
    for verified in verified_list:
        print(f"Seed File '{seed_name}': Seed Column '{verified['seed_column']}' -> Input Column '{verified['input_column']}' (Ratio: {verified['ratio']:.2f})")



Mapping Attempt: 'druva_seed:S_no' -> 'Document Number'
Input Unique Values (Document Number): ['000002568431040', '000002568431046', '000002568431047', '000002568431052', '001/2024-25', '002425-000386', '002425-000387', '002425-000388', '002425-000390', '002425-000391', '002425-000392', '002425-000393', '002425-000394', '002425-000395', '002425-000396', '002425-000397', '002425-000398', '002425-000399', '002425-000400', '002425-000401', '002425-000402', '002425-000403', '002425-000404', '002425-000405', '002425-000406', '002425-000407', '002425-000408', '002425-000409', '002425-000410', '002425-000411', '002425-000412', '002425-000413', '002425-000414', '002425-000415', '009/24-25', '016471510232114', '019', '085-2425', '100/24-25', '100424281612', '100624886644', '1007242631962', '1007242632281', '1007242632455', '1007243304659', '1008241043731', '1008243004813', '1009241967354', '1009242326077', '1009243072542', '102/24-25', '115', '151118137', '154', '155', '185', '186', '18958', 